# Write SPECFEM3D CMTSOLUTION and STATION files for each EVENT
1. Unpickle events and staions
1. Read and map moment tensors to events
1. Setup and test directory and file names
1. Write CMTSOLUTION and STATION files in separate dictories for each event

#### 1. Unpickle events and stations 

In [2]:
import pickle
from gnam.events.gevents import gevents as gevents
from gnam.events.gstations import gstations as gstations
from gnam.events.munge.knmi import correct_station_depths as csd_f

#Unpickle events
print('Unpickling Events')
f = open('../pickled/events.pickle', 'rb')
dill_events = pickle.load(f)
f.close()
    

#Unpickle stations
print('Unpickling Station Traces')
f = open('../pickled/straces.pickle', 'rb')
dill_straces = pickle.load(f)
f.close()

# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
dill_straces.correct_stations(csd_f)

print('Done')

Unpickling Events
Unpickling Station Traces
Done


#### 2. Read and map moment tensors to events

In [2]:
from gnam.events.mtensors import mtensors
import pandas as pd
import numpy as np

#Read moment tensors
gf_mts = mtensors('../data/event_moments.csv')
    
# get event catalog of the events withing the bounding box
e_cat = dill_events.getIncCatalog()

# This is a bit hokey, but it works. Here we update the
# event time from the moment tensor CSV file with thouse
# from the event catalog
gf_mts.update_utcdatetime(e_cat)

# Create a dictionary that maps moment tensors to events
e2mt_dict = gf_mts.map_events_2_tensors(e_cat)
e2mt_keys = e2mt_dict.keys()

# Print a comparison of events to moment tensors
for key in e2mt_keys:
    print('UTC: event[%d][Date] = %s' %(key,e_cat[key].origins[0].time))
    print('UTC:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['Date']))
    print('Mag: event[%d][Date] = %s' %(key,e_cat[key].magnitudes[0].mag))
    print('Mag:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['ML']))
    print()

UTC: event[15][Date] = 2016-02-25T22:26:30.899999Z
UTC:    MT[15][Date] = 2016-02-25T22:26:30.899999Z
Mag: event[15][Date] = 2.420941255
Mag:    MT[15][Date] = 2.4

UTC: event[14][Date] = 2016-09-02T13:16:00.900000Z
UTC:    MT[14][Date] = 2016-09-02T13:16:00.900000Z
Mag: event[14][Date] = 2.066445551
Mag:    MT[14][Date] = 2.1

UTC: event[13][Date] = 2016-11-01T00:57:46.000000Z
UTC:    MT[13][Date] = 2016-11-01T00:57:46.000000Z
Mag: event[13][Date] = 2.180530746
Mag:    MT[13][Date] = 2.2

UTC: event[12][Date] = 2017-03-11T12:52:48.000000Z
UTC:    MT[12][Date] = 2017-03-11T12:52:48.000000Z
Mag: event[12][Date] = 2.083340996
Mag:    MT[12][Date] = 2.1

UTC: event[11][Date] = 2017-04-26T13:56:49.299999Z
UTC:    MT[11][Date] = 2017-04-26T13:56:49.299999Z
Mag: event[11][Date] = 2.005890463
Mag:    MT[11][Date] = 2.0

UTC: event[10][Date] = 2017-05-27T15:29:00.600000Z
UTC:    MT[10][Date] = 2017-05-27T15:29:00.600000Z
Mag: event[10][Date] = 2.585955131
Mag:    MT[10][Date] = 2.6

UTC: event

#### 3. Setup and test directory names/paths

In [3]:
experiments = []
for ekey in e2mt_keys:
    bkeys = dill_straces.getBoreholeKeys()
    bh_stat_list = []
    for bkey in bkeys:
        bh_stat_list.append(dill_straces.getIncludedStations(ekey,bkey))
    bh_stat_tup = tuple(bh_stat_list)   
    exp_set = {'EKEY':ekey,'EVENT':e_cat[ekey],'TENSOR':e2mt_dict[ekey],'STATIONS':bh_stat_tup}
    experiments.append(exp_set)

dir_set_list = []
for eset in experiments:
    fqn = '../SPECFEM3D/events/obs'
    pid = 'EVENT%s' %(str(eset['EKEY']).zfill(3))
    nid = 'NL'
    y = eset['EVENT'].origins[0].time.year
    m = eset['EVENT'].origins[0].time.month
    d = eset['EVENT'].origins[0].time.day
    dname = '%s/%s_%s_%s_%02d_%02d' %(fqn,pid,nid,y,m,d)
    dir_set_list.append((eset,dname))
    print('Experiment-%d:\n'%(eset['EKEY']), dname)
    print()

Experiment-15:
 ../SPECFEM3D/events/obs/EVENT015_NL_2016_02_25

Experiment-14:
 ../SPECFEM3D/events/obs/EVENT014_NL_2016_09_02

Experiment-13:
 ../SPECFEM3D/events/obs/EVENT013_NL_2016_11_01

Experiment-12:
 ../SPECFEM3D/events/obs/EVENT012_NL_2017_03_11

Experiment-11:
 ../SPECFEM3D/events/obs/EVENT011_NL_2017_04_26

Experiment-10:
 ../SPECFEM3D/events/obs/EVENT010_NL_2017_05_27

Experiment-9:
 ../SPECFEM3D/events/obs/EVENT009_NL_2017_12_10

Experiment-8:
 ../SPECFEM3D/events/obs/EVENT008_NL_2018_01_08

Experiment-7:
 ../SPECFEM3D/events/obs/EVENT007_NL_2018_02_08

Experiment-6:
 ../SPECFEM3D/events/obs/EVENT006_NL_2018_02_11

Experiment-5:
 ../SPECFEM3D/events/obs/EVENT005_NL_2018_04_13

Experiment-4:
 ../SPECFEM3D/events/obs/EVENT004_NL_2019_05_22

Experiment-3:
 ../SPECFEM3D/events/obs/EVENT003_NL_2019_06_09



#### 4. Write CMTSOLUTION and STATION files in separate dictories for each event

In [4]:
from gnam.specutils.write import write_cmtsolution as spec_wcmt
from gnam.specutils.write import write_stations as spec_ws
import os

# define the access rights
access_rights = 0o755

#loop over dir-set pairs
for pair in dir_set_list:
    eset  = pair[0]
    dname = pair[1]
    print('Experiment-%d:\n'%(eset['EKEY']), dname)
    try:
        os.makedirs(dname, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % dname)
        print()
    else:
        bkeys = dill_straces.getBoreholeKeys()
        lxy = dill_events.getLocalEventCoord(eset['EKEY'])
        spec_wcmt(dname + '/CMTSOLUTION',eset['TENSOR'],lxy[0],lxy[1])
        spec_ws(dname + '/STATIONS',dill_straces,eset['EKEY'],bkeys)
        print ("Successfully created the directory %s" % dname)
        print()

Experiment-15:
 ../SPECFEM3D/events/obs/EVENT015_NL_2016_02_25
Successfully created the directory ../SPECFEM3D/events/obs/EVENT015_NL_2016_02_25

Experiment-14:
 ../SPECFEM3D/events/obs/EVENT014_NL_2016_09_02
Successfully created the directory ../SPECFEM3D/events/obs/EVENT014_NL_2016_09_02

Experiment-13:
 ../SPECFEM3D/events/obs/EVENT013_NL_2016_11_01
Successfully created the directory ../SPECFEM3D/events/obs/EVENT013_NL_2016_11_01

Experiment-12:
 ../SPECFEM3D/events/obs/EVENT012_NL_2017_03_11
Successfully created the directory ../SPECFEM3D/events/obs/EVENT012_NL_2017_03_11

Experiment-11:
 ../SPECFEM3D/events/obs/EVENT011_NL_2017_04_26
Successfully created the directory ../SPECFEM3D/events/obs/EVENT011_NL_2017_04_26

Experiment-10:
 ../SPECFEM3D/events/obs/EVENT010_NL_2017_05_27
Successfully created the directory ../SPECFEM3D/events/obs/EVENT010_NL_2017_05_27

Experiment-9:
 ../SPECFEM3D/events/obs/EVENT009_NL_2017_12_10
Successfully created the directory ../SPECFEM3D/events/obs/EVE

## Finished